In [3]:
# ==============================================
# End-to-end (4 subgroup models: Gender × AgeBin) + BMI feature + Optuna tuning
# Load → Detect ID/Target/Gender → Drop MTRANS/SMOKE → +BMI →
# Round Age → AgeBin (0:<24, 1:>=24) →
# Split to 4 groups → Per-group 5-Fold XGB (ES) [+ optional Optuna] →
# Predict → submission.csv
# ==============================================

# -------- Imports --------
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.base import clone

import xgboost as xgb
import optuna  # <- for hyperparameter tuning

# -------- Paths --------
TRAIN_PATH = "train.csv"
TEST_PATH = "test.csv"
SAMPLE_SUB_PATH = "sample_submission.csv"

RANDOM_STATE = 42
N_FOLDS = 5
N_JOBS = -1
NUM_CLASSES_EXPECTED = 7   # sanity warning only

# -------- Tuning toggles --------
USE_OPTUNA_TUNING = True     # set False to skip tuning and use base params
OPTUNA_TRIALS = 40           # per subgroup; increase for more thorough search

# -------- Helpers --------
def norm_col(s: str) -> str:
    if s is None: return s
    return str(s).replace("\ufeff", "").strip().lower()

def build_norm_map(cols):
    fwd = {c: norm_col(c) for c in cols}
    rev = {}
    for orig, n in fwd.items():
        if n not in rev:
            rev[n] = orig
    return fwd, rev

def find_id_and_label(sample_sub, train, test):
    ss_fwd, _ = build_norm_map(sample_sub.columns)
    tr_fwd, _ = build_norm_map(train.columns)
    te_fwd, _ = build_norm_map(test.columns)

    ss_norm_cols = [ss_fwd[c] for c in sample_sub.columns]
    tr_norm_cols = [tr_fwd[c] for c in train.columns]
    te_norm_cols = [te_fwd[c] for c in test.columns]

    id_norm, label_norm = None, None
    if len(ss_norm_cols) == 2:
        c1, c2 = ss_norm_cols
        if c1 in te_norm_cols and c2 not in te_norm_cols:
            id_norm, label_norm = c1, c2
        elif c2 in te_norm_cols and c1 not in te_norm_cols:
            id_norm, label_norm = c2, c1
        else:
            if c1 in te_norm_cols and c1 in tr_norm_cols:
                id_norm, label_norm = c1, c2
            elif c2 in te_norm_cols and c2 in tr_norm_cols:
                id_norm, label_norm = c2, c1

    if id_norm is None:
        for cand in ["id", "row_id", "index", "sample_id"]:
            if cand in te_norm_cols and cand in tr_norm_cols:
                id_norm = cand
                break

    if label_norm is None:
        candidates = [c for c in ss_norm_cols if c != id_norm]
        if len(candidates) == 1:
            label_norm = candidates[0]

    if label_norm is None:
        for cand in ["label", "target", "class", "y", "weightcategory", "nobeyesdad"]:
            if cand in tr_norm_cols and cand != id_norm:
                label_norm = cand
                break

    if label_norm is None:
        for c in reversed(tr_norm_cols):
            if c != id_norm:
                label_norm = c
                break

    return {
        "id_norm": id_norm,
        "label_norm": label_norm,
        "id_in_train": build_norm_map(train.columns)[1].get(id_norm, None),
        "id_in_test": build_norm_map(test.columns)[1].get(id_norm, None),
        "id_in_sample": build_norm_map(sample_sub.columns)[1].get(id_norm, None),
        "label_in_train": build_norm_map(train.columns)[1].get(label_norm, None),
        "label_in_sample": build_norm_map(sample_sub.columns)[1].get(label_norm, None),
    }

def infer_feature_types(df):
    cat_cols = df.select_dtypes(include=["object", "category", "bool"]).columns.tolist()
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    return num_cols, cat_cols

def detect_gender_column(df):
    candidates = [c for c in df.columns if norm_col(c) in {"gender","sex"}]
    if candidates:
        return candidates[0]
    for c in df.columns:
        vals = pd.Series(df[c].dropna().astype(str).str.lower().str.strip()).unique()
        if len(vals) in (2, 3):
            if any(v.startswith("m") for v in vals) and any(v.startswith("f") for v in vals):
                return c
    return None

def split_by_gender(series):
    s = series.astype(str).str.lower().str.strip()
    male_mask = s.str.startswith(("m","1","true"))
    female_mask = s.str.startswith(("f","0","false"))
    if male_mask.sum()==0 and female_mask.sum()==0:
        top = s.value_counts().index.tolist()
        if len(top)>=2:
            male_mask = s==top[0]
            female_mask = s==top[1]
    return male_mask, female_mask

def add_bmi(df):
    """Compute BMI exactly as Weight / (Height ** 2)."""
    if ("Weight" in df.columns) and ("Height" in df.columns):
        with np.errstate(divide="ignore", invalid="ignore"):
            df["BMI"] = pd.to_numeric(df["Weight"], errors="coerce") / (
                pd.to_numeric(df["Height"], errors="coerce") ** 2
            )
        df["BMI"] = df["BMI"].replace([np.inf, -np.inf], np.nan)
    return df

def round_and_bin_age(df):
    """Round Age, then make AgeBin = 0 if <24, 1 if >=24."""
    if "Age" in df.columns:
        age_num = pd.to_numeric(df["Age"], errors="coerce").round()
        df["Age"] = age_num
        df["AgeBin"] = (age_num >= 24).astype("Int8")
    return df

# --- Optuna-powered XGB tuner (CV + early stopping, macro F1) ---
def tune_xgb_hyperparams(
    X_df: pd.DataFrame,
    y_enc: np.ndarray,
    classes: list,
    n_trials: int = 40,
    n_folds: int = 5,
    random_state: int = 42,
    n_jobs: int = -1,
):
    """
    Returns best_params (dict) to merge into your xgb_params.
    Works with sparse outputs from ColumnTransformer+OHE.
    """
    num_cols, cat_cols = infer_feature_types(X_df)

    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler(with_mean=False))
    ])
    try:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
    except TypeError:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)
    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", ohe)
    ])
    preprocessor_tpl = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_cols),
            ("cat", categorical_transformer, cat_cols),
        ],
        remainder="drop",
        sparse_threshold=1.0
    )

    base = {
        "objective": "multi:softprob",
        "num_class": len(classes),
        "eval_metric": "mlogloss",
        "tree_method": "hist",
        "nthread": n_jobs,
        "seed": random_state,
    }
    NUM_BOOST_ROUND = 20000
    EARLY_STOP = 200
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_state)

    def objective(trial: optuna.Trial):
        params = {
            **base,
            "eta": trial.suggest_float("eta", 0.01, 0.06, log=True),
            "max_depth": trial.suggest_int("max_depth", 4, 12),
            "min_child_weight": trial.suggest_float("min_child_weight", 1.0, 10.0),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "lambda": trial.suggest_float("lambda", 1e-3, 10.0, log=True),
            "alpha": trial.suggest_float("alpha", 1e-4, 1.0, log=True),
            "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        }

        f1s = []
        for tr_idx, va_idx in skf.split(X_df, y_enc):
            X_tr, X_va = X_df.iloc[tr_idx], X_df.iloc[va_idx]
            y_tr, y_va = y_enc[tr_idx], y_enc[va_idx]

            prep = clone(preprocessor_tpl)
            Xtr = prep.fit_transform(X_tr)
            Xva = prep.transform(X_va)

            dtrain = xgb.DMatrix(Xtr, label=y_tr)
            dval   = xgb.DMatrix(Xva, label=y_va)

            bst = xgb.train(
                params=params,
                dtrain=dtrain,
                num_boost_round=NUM_BOOST_ROUND,
                evals=[(dtrain, "train"), (dval, "valid")],
                early_stopping_rounds=EARLY_STOP,
                verbose_eval=False
            )
            best_round = int(bst.best_iteration + 1)

            proba = bst.predict(dval, iteration_range=(0, best_round))
            pred = np.argmax(proba, axis=1)
            f1s.append(f1_score(y_va, pred, average="macro"))

            trial.report(float(np.mean(f1s)), step=len(f1s))
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return float(np.mean(f1s))

    study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=random_state))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)

    print(f"[Optuna] Best macro-F1: {study.best_value:.4f}")
    print(f"[Optuna] Best params: {study.best_params}")
    return study.best_params

# -------- Load data --------
train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sample_sub = pd.read_csv(SAMPLE_SUB_PATH)

# Drop not-used columns
for c in ["MTRANS","SMOKE"]:
    if c in train.columns: train.drop(columns=[c], inplace=True)
    if c in test.columns:  test.drop(columns=[c], inplace=True)

# >>> Add BMI + Round Age + AgeBin (train & test) <<<
train = add_bmi(train)
test  = add_bmi(test)
train = round_and_bin_age(train)
test  = round_and_bin_age(test)

info = find_id_and_label(sample_sub, train, test)

ID_COL_TRAIN   = info["id_in_train"]
ID_COL_TEST    = info["id_in_test"]
ID_COL_SAMPLE  = info["id_in_sample"]
TARGET_COL     = info["label_in_train"]
LABEL_COL_SAMP = info["label_in_sample"]

if TARGET_COL is None:
    raise ValueError("Could not detect the target column. Please ensure sample_submission and train headers align.")
if LABEL_COL_SAMP is None:
    ss_cols = list(sample_sub.columns)
    others = [c for c in ss_cols if c != ID_COL_SAMPLE]
    if len(others)==1:
        LABEL_COL_SAMP = others[0]
    else:
        raise ValueError("Could not detect label header in sample_submission.csv")

print(f"[Detected] Target in train: '{TARGET_COL}', Label in sample_sub: '{LABEL_COL_SAMP}'")
if ID_COL_TRAIN and ID_COL_TEST:
    print(f"[Detected] ID in train: '{ID_COL_TRAIN}', ID in test: '{ID_COL_TEST}'")

# -------- Target / Features --------
y = train[TARGET_COL].copy()
X = train.drop(columns=[TARGET_COL]).copy()
if ID_COL_TRAIN in X.columns:
    X.drop(columns=[ID_COL_TRAIN], inplace=True)

test_features = test.copy()
if ID_COL_TEST in test_features.columns:
    test_ids = test_features[ID_COL_TEST].copy()
    test_features.drop(columns=[ID_COL_TEST], inplace=True)
else:
    test_ids = pd.Series(np.arange(len(test_features)), name="id")

# -------- Label encode target --------
le = LabelEncoder()
y_enc = le.fit_transform(y)
classes = list(le.classes_)
if len(classes) != NUM_CLASSES_EXPECTED:
    print(f"[Warn] Expected {NUM_CLASSES_EXPECTED} classes but found {len(classes)}. Proceeding.")

# -------- Detect gender column and split (then 4-way split with AgeBin) --------
gender_col = detect_gender_column(pd.concat([X, test_features], axis=0))
if gender_col is None:
    raise ValueError("Could not detect a gender column (e.g., 'Gender' or 'SEX'). Please confirm the column name.")
if "AgeBin" not in X.columns:
    raise ValueError("AgeBin not found. Check round_and_bin_age step.")

male_mask, female_mask = split_by_gender(train[gender_col])
test_male_mask, test_female_mask = split_by_gender(test_features[gender_col])

young_mask = (train["AgeBin"] == 0)
old_mask   = (train["AgeBin"] == 1)
test_young_mask = (test_features["AgeBin"] == 0)
test_old_mask   = (test_features["AgeBin"] == 1)

print(f"[Info] Train counts - M<24:{int((male_mask & young_mask).sum())}  M>=24:{int((male_mask & old_mask).sum())}  F<24:{int((female_mask & young_mask).sum())}  F>=24:{int((female_mask & old_mask).sum())}")
print(f"[Info] Test  counts - M<24:{int((test_male_mask & test_young_mask).sum())}  M>=24:{int((test_male_mask & test_old_mask).sum())}  F<24:{int((test_female_mask & test_young_mask).sum())}  F>=24:{int((test_female_mask & test_old_mask).sum())}")

# We drop split columns (gender, Age, AgeBin) inside each subgroup (constants after split)
DROP_IN_GROUP = set([gender_col, "Age", "AgeBin"])

def make_preprocessor(Xdf):
    num_cols, cat_cols = infer_feature_types(Xdf)
    numeric_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler(with_mean=False))
    ])
    try:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
    except TypeError:
        ohe = OneHotEncoder(handle_unknown="ignore", sparse=True)
    categorical_transformer = Pipeline(steps=[
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("onehot", ohe)
    ])
    return ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_cols),
            ("cat", categorical_transformer, cat_cols),
        ],
        remainder="drop",
        sparse_threshold=1.0
    )

def train_group_and_predict(X_grp, y_enc_grp, test_grp, group_name, base_params, tune=False):
    # remove split columns from features if present
    cols_to_use = [c for c in X_grp.columns if c not in DROP_IN_GROUP]
    Xg = X_grp[cols_to_use].copy()
    Xtestg = test_grp[cols_to_use].copy()

    # optionally tune
    tuned = {}
    if tune and len(Xg) > 0 and len(np.unique(y_enc_grp)) > 1:
        tuned = tune_xgb_hyperparams(
            Xg, y_enc_grp, classes,
            n_trials=OPTUNA_TRIALS,
            n_folds=N_FOLDS,
            random_state=RANDOM_STATE,
            n_jobs=N_JOBS
        )

    # final params
    xgb_params = base_params.copy()
    xgb_params.update(tuned)

    preprocessor = make_preprocessor(Xg)

    NUM_BOOST_ROUND = 20000
    EARLY_STOP = 200
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

    oof_group = np.zeros((len(Xg), len(classes)), dtype=np.float32)
    test_group_pred = np.zeros((len(Xtestg), len(classes)), dtype=np.float32)
    fold_best = []

    for fold, (tr_idx, va_idx) in enumerate(skf.split(Xg, y_enc_grp), start=1):
        print(f"\n[{group_name}] Fold {fold}/{N_FOLDS}")
        X_tr, X_va = Xg.iloc[tr_idx], Xg.iloc[va_idx]
        y_tr, y_va = y_enc_grp[tr_idx], y_enc_grp[va_idx]

        prep = clone(preprocessor)
        Xtr = prep.fit_transform(X_tr)
        Xva = prep.transform(X_va)

        dtrain = xgb.DMatrix(Xtr, label=y_tr)
        dval   = xgb.DMatrix(Xva, label=y_va)

        bst = xgb.train(
            params=xgb_params,
            dtrain=dtrain,
            num_boost_round=NUM_BOOST_ROUND,
            evals=[(dtrain, "train"), (dval, "valid")],
            early_stopping_rounds=EARLY_STOP,
            verbose_eval=False
        )
        best_round = int(bst.best_iteration + 1)
        fold_best.append(best_round)
        print(f"[{group_name}] Best iteration: {best_round}")

        oof_proba = bst.predict(dval, iteration_range=(0, best_round))
        oof_group[va_idx] = oof_proba

        Xtest_tf = prep.transform(Xtestg)
        dtest = xgb.DMatrix(Xtest_tf)
        test_group_pred += bst.predict(dtest, iteration_range=(0, best_round)) / N_FOLDS

    # OOF summary for the group
    if len(Xg) > 0:
        oof_argmax = np.argmax(oof_group, axis=1)
        acc_g = accuracy_score(y_enc_grp, oof_argmax)
        f1_g = f1_score(y_enc_grp, oof_argmax, average="macro")
        print(f"\n[{group_name}] OOF Accuracy: {acc_g:.4f} | Macro F1: {f1_g:.4f}")
        if fold_best:
            print(f"[{group_name}] Best iterations: {fold_best} | Median: {int(np.median(fold_best))}")

    return oof_group, test_group_pred

# -------- Base XGBoost params (merged with tuned params if enabled) --------
BASE_XGB_PARAMS = {
    "objective": "multi:softprob",
    "num_class": len(classes),
    "eval_metric": "mlogloss",
    "tree_method": "hist",
    "max_depth": 6,
    "min_child_weight": 2.0,
    "subsample": 0.9,
    "colsample_bytree": 0.9,
    "lambda": 1.0,
    "alpha": 0.0,
    "eta": 0.03,
    "nthread": N_JOBS,
    "seed": RANDOM_STATE,
}

# -------- Build 4 groups and train --------
groups = [
    ("MALE_<24",   (male_mask & (train["AgeBin"] == 0)),   (test_male_mask & (test_features["AgeBin"] == 0))),
    ("MALE_>=24",  (male_mask & (train["AgeBin"] == 1)),   (test_male_mask & (test_features["AgeBin"] == 1))),
    ("FEMALE_<24", (female_mask & (train["AgeBin"] == 0)), (test_female_mask & (test_features["AgeBin"] == 0))),
    ("FEMALE_>=24",(female_mask & (train["AgeBin"] == 1)), (test_female_mask & (test_features["AgeBin"] == 1))),
]

oof_full = np.zeros((len(X), len(classes)), dtype=np.float32)
test_pred_proba = np.zeros((len(test_features), len(classes)), dtype=np.float32)

for grp_name, tr_mask, te_mask in groups:
    if tr_mask.sum() == 0:
        print(f"[Warn] No training rows for group {grp_name}; skipping.")
        continue

    X_grp = X[tr_mask].reset_index(drop=True)
    y_grp = y_enc[tr_mask.values]
    test_grp = test_features[te_mask].reset_index(drop=True)

    grp_oof, grp_test_pred = train_group_and_predict(
        X_grp, y_grp, test_grp, grp_name,
        base_params=BASE_XGB_PARAMS,
        tune=USE_OPTUNA_TUNING
    )

    oof_full[tr_mask.values] = grp_oof
    test_pred_proba[te_mask.values] = grp_test_pred

# -------- Overall OOF report --------
oof_labels = np.argmax(oof_full, axis=1)
oof_acc = accuracy_score(y_enc, oof_labels)
oof_f1 = f1_score(y_enc, oof_labels, average="macro")
print("\n========== OVERALL OOF ==========")
print(f"OOF Accuracy: {oof_acc:.4f} | OOF Macro F1: {oof_f1:.4f}")
try:
    print("\nOOF Classification Report:\n",
          classification_report(y_enc, oof_labels, target_names=classes))
except Exception as e:
    print(f"[Info] Could not print classification report: {e}")

# -------- Build submission --------
test_pred_int = np.argmax(test_pred_proba, axis=1)
test_pred_labels = le.inverse_transform(test_pred_int)

ss_cols = list(sample_sub.columns)
ID_HEADER = ID_COL_SAMPLE if ID_COL_SAMPLE in sample_sub.columns else None
LABEL_HEADER = LABEL_COL_SAMP

sub = pd.DataFrame()
if ID_HEADER is not None and ID_COL_TEST in test.columns:
    sub[ID_HEADER] = test[ID_COL_TEST].values
elif ID_HEADER is not None:
    sub[ID_HEADER] = np.arange(len(test_features))
sub[LABEL_HEADER] = test_pred_labels

# Reorder/complete to match sample_sub exactly
for c in ss_cols:
    if c not in sub.columns:
        sub[c] = sample_sub[c].iloc[0] if len(sample_sub[c]) else None
sub = sub[ss_cols]

sub.to_csv("submission.csv", index=False)
print("\nSaved submission.csv")
print(sub.head(10))


[I 2025-10-23 17:16:42,501] A new study created in memory with name: no-name-832cad8e-c6a7-4acc-8395-19d88e6dee7b


[Detected] Target in train: 'WeightCategory', Label in sample_sub: 'WeightCategory'
[Detected] ID in train: 'id', ID in test: 'id'
[Info] Train counts - M<24:4551  M>=24:3232  F<24:4755  F>=24:2995
[Info] Test  counts - M<24:595  M>=24:471  F<24:643  F>=24:400


C:\Users\surya\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
[I 2025-10-23 17:17:28,492] Trial 0 finished with value: 0.7929147888052066 and parameters: {'eta': 0.01956360392823701, 'max_depth': 12, 'min_child_weight': 7.587945476302646, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'lambda': 0.004207053950287938, 'alpha': 0.0001707396743152812, 'gamma': 4.330880728874676}. Best is trial 0 with value: 0.7929147888052066.
C:\Users\surya\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(
[I 2025-10-23 17:17:50,469] Trial 1 finished with value: 0.7963879322196303 and parameters: {'eta': 0.0293601586511158, 'max_depth': 10, 'min_child_weight': 1.185260448662222, 'subsam

[Optuna] Best macro-F1: 0.8000
[Optuna] Best params: {'eta': 0.017248307328108965, 'max_depth': 8, 'min_child_weight': 4.887505167779041, 'subsample': 0.7164916560792167, 'colsample_bytree': 0.8447411578889518, 'lambda': 0.003613894271216527, 'alpha': 0.0014742753159914669, 'gamma': 1.8318092164684585}

[MALE_<24] Fold 1/5
[MALE_<24] Best iteration: 2500

[MALE_<24] Fold 2/5
[MALE_<24] Best iteration: 3439

[MALE_<24] Fold 3/5
[MALE_<24] Best iteration: 2134

[MALE_<24] Fold 4/5
[MALE_<24] Best iteration: 4422

[MALE_<24] Fold 5/5


[I 2025-10-23 17:27:01,958] A new study created in memory with name: no-name-072b55d3-37f0-414a-9e46-2a9751a1661f


[MALE_<24] Best iteration: 1611

[MALE_<24] OOF Accuracy: 0.8699 | Macro F1: 0.7504
[MALE_<24] Best iterations: [2500, 3439, 2134, 4422, 1611] | Median: 2500


C:\Users\surya\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
[I 2025-10-23 17:27:28,558] Trial 0 finished with value: 0.6343277042005868 and parameters: {'eta': 0.01956360392823701, 'max_depth': 12, 'min_child_weight': 7.587945476302646, 'subsample': 0.8394633936788146, 'colsample_bytree': 0.6624074561769746, 'lambda': 0.004207053950287938, 'alpha': 0.0001707396743152812, 'gamma': 4.330880728874676}. Best is trial 0 with value: 0.6343277042005868.
C:\Users\surya\AppData\Roaming\Python\Python310\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
[I 2025-10-23 17:27:39,652] Trial 1 finished with value: 0.6355770343994949 and parameters: {'eta': 0.0293601586511158, 'max_depth': 10, 'min_child_weight': 1.185260448662222, 'subsam

KeyboardInterrupt: 